In [6]:
import student_generator as sg
import numpy as np
import names as nm
import pandas as pd
import tensorflow as tf

# 1. Generate students

students = sg.generate_students(60)[:,:4]
student_surnames = np.array([nm.get_random_surname() for i in range(1500)])
student_names = np.array([nm.get_random_name() for i in range(1500)])
student_patronymics = np.array([nm.get_random_patronymic() for i in range(1500)])

# 2. Create dataframe with students and their names

df = pd.DataFrame(student_surnames, columns=['Surname'])
df['Name'] = student_names
df['Patronymic'] = student_patronymics
df['Math'] = students[:,0].round(0)
df['English'] = students[:,1].round(0)
df['Ukrainian'] = students[:,2].round(0)
df['Privilege'] = ["Yes" if i == 1 else "No" for i in students[:,3]]

# 3. Save dataframe to excel file

df.to_excel('students.xlsx', index=False)





In [8]:
# Read data from excel file

df = pd.read_excel('students.xlsx')

# Convert privilege column to 0 and 1

df['Privilege'] = [1 if i == "Yes" else 0 for i in df['Privilege']]

# Convert to numpy array and format data

students = df.to_numpy()
students = students[:,3:7]
students = students.astype('float32')


In [11]:
model = tf.keras.models.load_model('models/model-1500-1500-350-60.h5')

# Predict results

results = model.predict(students)
results = results.reshape(1500).round(0)


47/47 [==============================] - 0s 3ms/step
[1. 0. 0. ... 0. 0. 0.]


In [32]:
accepted_df = pd.DataFrame(df['Surname'], columns=['Surname'])
accepted_df['Name'] = df['Name']
accepted_df['Patronymic'] = df['Patronymic']
accepted_df['Math'] = df['Math']
accepted_df['English'] = df['English']
accepted_df['Ukrainian'] = df['Ukrainian']
# privilege back to Yes/No
accepted_df['Privilege'] = ["Yes" if i == 1 else "No" for i in df['Privilege']]
accepted_df['Exam average'] = (df['Math'] * 0.4 + df['English']*  0.3 + df['Ukrainian'] * 0.3) 
accepted_df['Accepted'] = ["Yes" if i == 1 else "No" for i in results]

import openpyxl as xl
from openpyxl.utils.dataframe import dataframe_to_rows

# Create new excel file with accepted students

wb = xl.Workbook()
ws = wb.active

# Color student background green if he is accepted

for row in dataframe_to_rows(accepted_df, index=False, header=True):
    ws.append(row)
    if row[8] == "Yes":
        for cell in ws[ws.max_row]:
            cell.fill = xl.styles.PatternFill(start_color='BDFFC8', end_color='BDFFC8', fill_type='solid')
    else :
        for cell in ws[ws.max_row]:
            cell.fill = xl.styles.PatternFill(start_color='FFBDBD', end_color='FFBDBD', fill_type='solid')



wb.save('accepted_students.xlsx')